# Lineup Frequency

## Whenever ready click the run all button. If prompted to choose a server, it will be Python 3.12

![kernel](./misc/instru.png)

## Getting timeline as csv

### First open timeline and cmd + a for all instances.

![timeline](./misc/instru2.png)

### Then you go to File > Export > CSV File... and save to whatever folder
![export](./misc/instru3.png)



## The Code

#### This first block just imports libraries and no action is needed other than making sure it runs. In the future it is possible the libraries may need to be updated, just text me (Ankith) about it and I'll walk through it, it is an easy process.

In [25]:
import pandas as pd
import numpy as np
import re
import plotly.figure_factory as ff
import os


### This creates function to calculate the stats we want

In [26]:
# Define a function to sort the contents of each cell numerically
def sort_cell_contents(cell):
    if isinstance(cell, str):
        items = [x.strip() for x in cell.split(',')]
        items.sort(key=lambda x: int(re.search(r'\d+', x).group()))
        return ', '.join(items)
    else:
        return cell

# Define a function to calculate points based on the 'Result' value
def calculate_points(result):
    if result in ['O3', 'O3F']:
        return 3
    elif result in ['O2', 'O2F']:
        return 2
    elif result == 'FT - MK':
        return 1
    return 0

# Function to count the shots
def count_shots(result):
    return 1 if result in ['O2', 'O3', 'X2', 'X3', 'X3F', 'X2F', 'TO'] else 0

# Function to identify turnovers
def count_turnovers(result):
    return 1 if result == 'TO' else 0

# Function to count field goals made
def count_field_goals_made(result):
    return 1 if result in ['O2', 'O3', 'O2F', 'O3F'] else 0

# Function to count total field goals attempted
def count_field_goals_attempted(result):
    return 1 if result in ['O2', 'O3', 'X2', 'X3', 'O2F', 'O3F'] else 0

# Function to count three-pointers made
def count_three_point_made(result):
    return 1 if result in ['O3', 'O3F'] else 0

# Function to count total three-pointers attempted
def count_three_point_attempted(result):
    return 1 if result in ['O3', 'X3', 'O3F'] else 0

# Function to count free throws made
def count_free_throws_made(result):
    return 1 if result == 'FT - MK' else 0

# Function to count free throws attempted
def count_free_throws_attempted(result):
    return 1 if result in ['FT - MK', 'FT - MI'] else 0

# Function to count our offensive rebounds
def count_our_offensive_rebounds(result):
    return 1 if result == 'GT Off Reb' else 0

# Function to count opponent's offensive rebounds
def count_opponent_offensive_rebounds(result):
    return 1 if result == 'Opp Off Reb' else 0


### The code block below imports every game within the game-csv folder and gets the frequency of each lineup. For example in the future, if working with 2024 games, then it would be 
 > folder_path = 'game-csv-2024'

In [27]:
# Import CSV files from folder and calculate lineup frequencies
folder_path = 'game-csv-2023'  # Update for the year of interest
dataframes = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        df['ON COURT'] = df['ON COURT'].apply(sort_cell_contents)
        df['Points'] = df['Result'].apply(calculate_points)
        df['Shots'] = df['Result'].apply(count_shots)
        df['Turnovers'] = df['Result'].apply(count_turnovers)
        df['FG Made'] = df['Result'].apply(count_field_goals_made)
        df['FG Attempted'] = df['Result'].apply(count_field_goals_attempted)
        df['3P Made'] = df['Result'].apply(count_three_point_made)
        df['3P Attempted'] = df['Result'].apply(count_three_point_attempted)
        df['FT Made'] = df['Result'].apply(count_free_throws_made)
        df['FT Attempted'] = df['Result'].apply(count_free_throws_attempted)
        df['Our Off Reb'] = df['Result'].apply(count_our_offensive_rebounds)
        df['Opp Off Reb'] = df['Result'].apply(count_opponent_offensive_rebounds)
        dataframes.append(df)

# Combine all games into one DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)


### This creates a table of lineup frequencies, and from here you can use the lineup frequency as needed.

In [28]:
# Aggregate the summed stats for each lineup
lineup_stats = combined_df.groupby('ON COURT').agg({
    'Points': 'sum',
    'Shots': 'sum',
    'Turnovers': 'sum',
    'FG Made': 'sum',
    'FG Attempted': 'sum',
    '3P Made': 'sum',
    '3P Attempted': 'sum',
    'FT Made': 'sum',
    'FT Attempted': 'sum',
    'Our Off Reb': 'sum',
    'Opp Off Reb': 'sum'
}).reset_index()

# Calculate percentages and points per shot
lineup_stats['FG%'] = ((lineup_stats['FG Made'] / lineup_stats['FG Attempted']) * 100).round(2)
lineup_stats['3P%'] = ((lineup_stats['3P Made'] / lineup_stats['3P Attempted']) * 100).round(2)
lineup_stats['FT%'] = ((lineup_stats['FT Made'] / lineup_stats['FT Attempted']) * 100).round(2)
lineup_stats['Points per Shot'] = (lineup_stats['Points'] / lineup_stats['Shots']).round(2)
lineup_stats['Turnover Percentage'] = ((lineup_stats['Turnovers'] / (lineup_stats['Shots'] + lineup_stats['Turnovers'])) * 100).round(2)  # Assuming shots represent possessions
lineup_stats['Our Off Reb %'] = (lineup_stats['Our Off Reb'] / lineup_stats['Shots']).round(3) * 100
lineup_stats['Opp Off Reb %'] = (lineup_stats['Opp Off Reb'] / lineup_stats['Shots']).round(3) * 100

# Merge the aggregated stats with the frequency table
frequency_table = combined_df['ON COURT'].value_counts().reset_index()
frequency_table.columns = ['Lineup', 'Frequency']
final_table = frequency_table.merge(lineup_stats, left_on='Lineup', right_on='ON COURT')

# Select and order columns for the final table
final_table = final_table[['Lineup', 'Frequency', 'Points', 'Points per Shot', 'Turnovers', 'Turnover Percentage',
                           'FG Made', 'FG Attempted', 'FG%', '3P Made', '3P Attempted', '3P%', 'FT Made', 'FT Attempted', 'FT%',
                           'Our Off Reb', 'Our Off Reb %', 'Opp Off Reb', 'Opp Off Reb %']]

# Create a Plotly table
fig = ff.create_table(final_table)

# Adjust the layout for the figure
fig.update_layout(
    width=8500,  # Adjust the total figure width
    height=5000,  # Adjust the total figure height
    margin=dict(l=10, r=10, b=10, t=10)  # Adjust the figure margins if necessary
)

# Show the table
fig.show()

final_table.to_excel('lineup_data.xlsx', index=False)
